In [19]:
import numpy as np
import pandas as pd
import pickle
from datetime import datetime
from sqlalchemy import create_engine, MetaData, Table, Column, Integer, String, Float, Date
from sklearn.cluster import KMeans
from sklearn.preprocessing import PowerTransformer
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
import seaborn as sns
import yaml
from surprise import Dataset, Reader, SVD
import os

In [20]:
try:
    with open("/home/athel/Desktop/Litmus7/order/config.yaml", "r") as f:
        config = yaml.safe_load(f)
    
    # Extract all required paths from config
    CLUSTER_NUMBER = config["cluster_number"]
    MERGED_DATA_PATH = config["file_paths"]["merged_data"]
    CUSTOMER_DF_PATH = config["file_paths"]["customer_df"]
    KMEANS_MODEL_PATH = config["file_paths"]["kmeans_model"]
    PT_MODEL_PATH = config["file_paths"]["pt_model"]
    PCA_MODEL_PATH = config["file_paths"]["pca_model"]
    CF_MODEL_TEMPLATE = config["file_paths"]["cf_model_template"]
    NEW_MERGED_DATA_PATH = config["file_paths"]["new_merged_data"]
    ORDER_INFO_NORM = config["file_paths"]["order_info_norm"]
    ORDER_LINE_NORM = config["file_paths"]["order_line_norm"] 
    PRODUCT_NORM = config["file_paths"]["product_norm"]

except Exception as e:
    raise SystemExit(f"Error loading config: {str(e)}")



In [21]:
try:
    df = pd.read_csv(MERGED_DATA_PATH, parse_dates=['Date'])
    df['Date'] = pd.to_datetime(df['Date']).dt.date
except FileNotFoundError:
    df = pd.DataFrame()

In [22]:
if not df.empty:
    # Ensure numeric columns
    numeric_cols = ['Quantity', 'Price per Unit']
    df[numeric_cols] = df[numeric_cols].apply(pd.to_numeric, errors='coerce')

    # Ensure 'Date' is datetime and drop rows with invalid dates
    df['Date'] = pd.to_datetime(df['Date'], errors='coerce')
    df = df.dropna(subset=['Date']).copy()

    # Calculate Sales
    df['Sales'] = df['Quantity'] * df['Price per Unit']

    # Calculate Recency (in days)
    today = pd.Timestamp(datetime.now().date())
    df['Recency'] = (today - df['Date']).dt.days

    # Calculate Order Gap
    df = df.sort_values(['Customer ID', 'Date'])
    df['Order Gap'] = df.groupby('Customer ID')['Date'].diff().dt.days.fillna(0)


In [23]:
if not df.empty and os.path.exists(NEW_MERGED_DATA_PATH) and os.path.getsize(NEW_MERGED_DATA_PATH) > 0:
    try:
        new_df = pd.read_csv(NEW_MERGED_DATA_PATH, parse_dates=['Date'])
        new_df['Date'] = pd.to_datetime(new_df['Date']).dt.date
        existing_ids = df[['Order ID', 'Product ID', 'SKU ID']].drop_duplicates()
        new_df = new_df.merge(existing_ids, on=['Order ID', 'Product ID', 'SKU ID'], 
                             how='left', indicator=True)
        new_df = new_df[new_df['_merge'] == 'left_only'].drop('_merge', axis=1)
        if not new_df.empty:
            df = pd.concat([df, new_df], ignore_index=True)
            print(f"Added {len(new_df)} new records")
    except Exception as e:
        print(f"New data ingestion error: {str(e)}")

In [24]:
def create_normalized_tables():
    # Order Info
    order_info = df[['Order ID', 'Customer ID', 'Warehouse ID', 'Customer Age',
                    'Customer Gender', 'Date', 'Recency', 'Order Gap']].drop_duplicates()
    order_info.to_csv(ORDER_INFO_NORM, index=False)
    
    # Order Line
    order_line = df[['Order ID', 'Product ID', 'SKU ID', 'Category',
                    'Quantity', 'Price per Unit', 'Sales']].drop_duplicates()
    order_line.to_csv(ORDER_LINE_NORM, index=False)
    
    # Product
    product = df[['Product ID', 'SKU ID', 'Category', 'Price per Unit']].drop_duplicates()
    product.to_csv(PRODUCT_NORM, index=False)

create_normalized_tables()

In [25]:
customer_df = df.groupby('Customer ID').agg(
    total_spend=('Sales', 'sum'),
    purchase_frequency=('Order ID', 'nunique'),
    avg_basket_size=('Quantity', 'mean'),
    cat_diversity=('Category', 'nunique'),
    recency=('Date', lambda x: (datetime.now().date() - x.max().date()).days),
    gap=('Order Gap', 'mean'),
    age=('Customer Age', 'first')
).reset_index()

In [26]:
features = ['total_spend','purchase_frequency','avg_basket_size','cat_diversity','recency','gap','age']
X = customer_df[features].values

pt = PowerTransformer(method='yeo-johnson', standardize=True)
pca = PCA(n_components=2)
X_pca = pca.fit_transform(pt.fit_transform(X))

In [27]:
kmeans = KMeans(n_clusters=CLUSTER_NUMBER, random_state=42)
customer_df['KMeans Cluster'] = kmeans.fit_predict(X_pca)

cluster_cols = [col for col in df.columns if 'KMeans Cluster' in col]
df = df.drop(columns=cluster_cols, errors='ignore')

df = pd.merge(df, customer_df[['Customer ID', 'KMeans Cluster']], 
             on='Customer ID', how='left', validate='many_to_one')


# Save models
for obj, path in [(pt, PT_MODEL_PATH), (pca, PCA_MODEL_PATH), (kmeans, KMEANS_MODEL_PATH)]:
    with open(path, 'wb') as f:
        pickle.dump(obj, f)

In [28]:

# Collaborative Filtering per cluster
for cluster_id in customer_df['KMeans Cluster'].unique():
    cluster_users = customer_df[customer_df['KMeans Cluster'] == cluster_id]['Customer ID']
    cluster_data = df[df['Customer ID'].isin(cluster_users)]
    
    data_long = cluster_data[['Customer ID', 'Product ID', 'Quantity']]
    reader = Reader(rating_scale=(data_long['Quantity'].min(), data_long['Quantity'].max()))
    algo = SVD()
    algo.fit(Dataset.load_from_df(data_long, reader).build_full_trainset())
    
    with open(CF_MODEL_TEMPLATE.format(cluster_id=cluster_id), 'wb') as f:
        pickle.dump(algo, f)

In [29]:
customer_df.to_csv(CUSTOMER_DF_PATH, index=False)
df.to_csv(MERGED_DATA_PATH, index=False)
print("Pipeline completed successfully")
print(df.head())

Pipeline completed successfully
   Order ID   Product ID   SKU ID                Category  Quantity  \
0      4248  Product_187  SKU_484         Home Appliances         2   
1      4248  Product_056  SKU_689             Electronics         2   
2      4248  Product_009  SKU_234         Home Appliances         5   
3      4248  Product_126  SKU_651  Beauty & Personal Care         3   
4      4248  Product_040  SKU_534       Health & Wellness         1   

   Price per Unit    Sales Customer ID Warehouse ID  Customer Age  \
0          786.10  1572.20     CUST001        WH003            24   
1          864.04  1728.08     CUST001        WH003            24   
2          604.22  3021.10     CUST001        WH003            24   
3           28.26    84.78     CUST001        WH003            24   
4          203.58   203.58     CUST001        WH003            24   

  Customer Gender       Date  Recency  Order Gap  KMeans Cluster  
0          Female 2023-03-17      813        0.0           

In [30]:
df

,Order ID,Product ID,SKU ID,Category,Quantity,Price per Unit,Sales,Customer ID,Warehouse ID,Customer Age,Customer Gender,Date,Recency,Order Gap,KMeans Cluster
0,4248,Product_187,SKU_484,Home Appliances,2,786.10,1572.20,CUST001,WH003,24,Female,2023-03-17,813,0.0,0
1,4248,Product_056,SKU_689,Electronics,2,864.04,1728.08,CUST001,WH003,24,Female,2023-03-17,813,0.0,0
2,4248,Product_009,SKU_234,Home Appliances,5,604.22,3021.10,CUST001,WH003,24,Female,2023-03-17,813,0.0,0
3,4248,Product_126,SKU_651,Beauty & Personal Care,3,28.26,84.78,CUST001,WH003,24,Female,2023-03-17,813,0.0,0
4,4248,Product_040,SKU_534,Health & Wellness,1,203.58,203.58,CUST001,WH003,24,Female,2023-03-17,813,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
44792,402,Product_001,SKU_314,Health & Wellness,5,531.25,2656.25,CUST999,WH004,37,Female,2023-11-22,563,58.0,0
44793,402,Product_001,SKU_314,Health & Wellness,5,531.25,2656.25,CUST999,WH004,37,Female,2023-11-22,563,0.0,0
44794,402,Product_127,SKU_860,Books & Media,4,322.30,1289.20,CUST999,WH004,37,Female,2023-11-22,563,0.0,0
44795,402,Product_127,SKU_860,Books & Media,4,322.30,1289.20,CUST999,WH004,37,Female,2023-11-22,563,0.0,0
